# Contents
1. [Evaluate Fastq properties](#Evaluate Fastq properties)
+ [NGS data processing: mapping and optimizing the WES data.](#NGS data processing: mapping and optimizing the WES data.)
    1. [Mapping with BWA](#Mapping with BWA)
    + [Marking duplicates](#Marking duplicates)
    + [Adding ReadGroups](#Adding ReadGroups)
+ [NGS data processing: mapping and optimizing the WGS data](#NGS data processing: mapping and optimizing the WGS data)
+ [Variant Calling & filtering](#Variant Calling & filtering)
    1. [Variant Calling](#Variant Calling)
    + [Variant Filtering](#Variant Filtering)
    + [Variant Annotation](#Variant Annotation)

Exercise processing, visualization and interpretation of NGS data.
==================================================================

The raw output of a NGS sequencer is a FASTA file, which is a flat text file containing sequence reads and associated read names and quality scores on every line in the file.

![The Fastq Format](https://kscbioinformatics.files.wordpress.com/2017/02/rawilluminadatafastqfiles.png?w=840)

A first step in the analysis of next-generation sequencing reads concerns the mapping of the sequence reads to a reference genome. We have generated three types of sequencing data:
1.	Partial whole exome sequencing (WES): datafiles/na12878_wes_brcagenes-1.fastq **and** datafiles/na12878_wes_brcagenes-2.fastq
2.	Partial RNA-sequencing (RNA)
3.	Partial Whole genome sequencing (WGS): datafiles/na12878_wgs_brcagenes-1.fastq **and** datafiles/na12878_wgs_brcagenes-2.fastq

Each of these datasets were derived from a public human sample and the sequence reads must be mapped to the human reference genome. The output of the mapping is a so-called Binary Alignment Map (BAM) file, which is not readable, but can be visualized using the Integrative Genomics Viewer. 

<span style="color:red">***IMPORTANT: $filename markings indicates that you should change this into YOUR filename of either an existing file or a new filename.***</span>



1. Evaluate Fastq properties<a name="Evaluate Fastq properties"></a>
=============
A very well-known tool for evaluating the quality of sequence data is FastQC (http://www.bioinformatics.babraham.ac.uk/projects/fastqc). Since processing NGS data can be a compute intensive step, it could be advisable to check if the data is any good before proceeding.


<span style="color:green">[DO]: Run fastqc on the fastq files: ‘fastqc datafiles/*.fastq’ to generate a HTML report for all the fastq files in the folder datafiles.</span>

To create links to the resulting HTML pages you’ll need to take your amazon-computer name from your webbrowsers adressbar (in between with https:// and amazonaws.com) and copy it in the instructions below:

In [1]:
for i in datafiles/*.html; 
    do echo https://COMPUTERNAMEHERE.amazonaws.com:8888/files/$i;
done

https://COMPUTERNAMEHERE.amazonaws.com:8888/files/datafiles/na12878_wes_brcagenes-1_fastqc.html
https://COMPUTERNAMEHERE.amazonaws.com:8888/files/datafiles/na12878_wes_brcagenes-2_fastqc.html
https://COMPUTERNAMEHERE.amazonaws.com:8888/files/datafiles/na12878_wgs_brcagenes-1_fastqc.html
https://COMPUTERNAMEHERE.amazonaws.com:8888/files/datafiles/na12878_wgs_brcagenes-2_fastqc.html


<span style="color:green">[DO]: take 10’ to review the result pages. What is your opinion on  the quality of the data? If needed, compare it to an (old) example of poor data (http://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html) or good data (http://www.bioinformatics.babraham.ac.uk/projects/fastqc/good_sequence_short_fastqc.html).</span>

[A]:

2 NGS data processing: mapping and optimizing the WES data.<a name="NGS data processing: mapping and optimizing the WES data."></a>
==============================
2A Mapping with BWA<a name="Mapping with BWA"></a>
------------------
BWA needs a number of files to start the alignment: execute ‘bwa mem’ to see them. We will use default settings, but still at least 3 files are required.


<span style="color:purple">[Q]: What files are required to start the mapping?</span>

[A]:

<span style="color:purple">[Q]: What is the default ouput location of the resulting alignmentfile?</span>

[A]:

<span style="color:green">[DO]: start an alignment of the WES (!) sequence reads in paired end mode, and store the output file. Remember to name it properly so you can see from which experiment the alignment was.</span>

<span style="color:green">[DO]: show the first 30 lines of the SAM file you’ve obtained by executing 
```bash 
head -n 30 $file
```
where \$file is your samfile. You’ll clearly see a header section and the start of the reads section. </span>

<span style="color:purple">[Q]: What does this entry in the header mean?</span>

@SQ     SN:21   LN:48129895

[A]:

<span style="color:purple"> 
[Q]: Describe some read details from the sam file for the first read. If you need more explanation, see the format specifications [here.](https://samtools.github.io/hts-specs/SAMv1.pdf)
-	<span style="color:purple"> What is it’s orientation?
-	<span style="color:purple"> Where does it map to in the genome?
-	<span style="color:purple"> Does it have mismatches compared to the reference?
-	<span style="color:purple"> What is the distance to its mate?
</span>

[A]:

The SAM files are not a very efficient way to handle these kind of data: information can be compressed and indexed by converting the SAM (Sequence Alignment Map) file into a BAM (Binary alignment Map).

<span style="color:green">[DO]: Covert your SAM file into a BAM file with ‘sambamba view -S -f bam \$samfile > \$bamfile’, where \$samfile is your samfile and \$bamfile is the bamfile to be generated. Give it a proper name.<span>

<span style="color:purple"> [Q]: Compare the filesizes of the SAM and BAM file. What is the compression ratio?</span>

[A]:

To be able to make use of the indexed searching options, we need to sort the BAM file by coordinates (default). 

<span style="color:green">[DO]: Sort your BAM file by 
```bash
sambamba sort \$bamfile
```
<span style="color:green">This will result in a \$bamfile.sorted automatically.</span>

<span style="color:purple">[Q]: Review the headers of the sorted and unsorted $bamfile by executing 
```bash
sambamba view -H \$bamfile
``` 
<span style="color:purple">for both and compare. Notice the flag stating the bam file is sorted (and how).</span>

[A]:

### 2B Marking duplicates <a name="Marking duplicates"></a>

We need to mark (but not remove) the duplicate reads in our (sorted) bamfile so these won’t be used by analysis tools lateron.

<span style="color:green">[DO]: mark the duplicates by executing 
```bash
sambamba markdup \$sorted_bamfile \$dedupped_sorted_bamfile
```
</span>

<span style="color:purple">[Q]: what is the percentage duplicate reads in this dataset?</span>

[A]:

### 2C Adding ReadGroups <a name="Adding ReadGroups"></a>
Previously you’ve examined the header of the SAM file and probably noticed that it contains very usefull information. Remember, it is very important to know from what sequence data and genome the alignment was generated. Additionally, you would want to know what tools and settings were used to be able to replicate results (See later in the course the F.A.I.R. principles). In the REadGroups section of the header you can store information on the sample and experiment (see details [SAM specifications](http://samtools.github.io/hts-specs/SAMv1.pdf)). This is considered so important that GATK for instance requires additional information in these readgroups. In advanced analyses you can actively use this information from the readgroups.

In this part we are going to fill in the minimal required readgroups with data using PicardTools ($PICARD): AddOrReplaceReadGroups. In real life we advise to be much more elaborate and specific.
- LB (library): WES
- PL (platform): illumina
- PU (platform unit): slide_barcode
- SM (sample name): na12878


<span style="color:green">[DO]: Execute Picardtools AddOrReplaceReadGroups with the following parameters (adjust to your own BAM file) and the data above:
```
java -jar $PICARD AddOrReplaceReadGroups \
I=$inputfile \
O=$outputfile \
CREATE_INDEX=true \
RGLB=$LB \
RGPL=$PL \
RGSM=$SM \
RGPU=$PU \
```



### 3. NGS data processing: mapping and optimizing the WGS data <a name="NGS data processing: mapping and optimizing the WGS data"></a>
Now we successfully have performed the steps to align and optimize the results of the exome sequencing, we want to analyse the whole genome data(WGS) data in the same way.

<span style="color:green">[DO]: repeat the steps to get to a sorted, deduplicated BAM file with readgroups for the WGS fastqs. Take care not to overwrite your WES results, and clearly mark WGS in the appropriate readgroup.</span>


## 4. Variant Calling & filtering <a name="Variant Calling & filtering"></a>

### 4A. Variant Calling <a name="Variant Calling"></a>

We will use the [GenomeAnalysisToolkit](https://software.broadinstitute.org/gatk/) component HaploTypeCaller to call variants from our BAM files. Since we are using default settings, only three parameters are required: **R**:the Genome reference (the same file you used in the mapping procedure), **I**: the input BAM file (with the readgroups) and an **o**:output filename.

<span style="color:green">[DO]: Complete the code below to start the variantcaller on your WES dataset. It will take ~15 min to complete, but will show detailed information.</span>

In [ ]:
java -jar $GATK \
-T HaplotypeCaller \
-R  \
-I  \
-o 

<span style="color:green">[DO]: Repeat this step to call variants on the WGS dataset. Runtime will be similar.</span>

If everything went well you should now have two vcf files with variants from the WES and WGS datasets.Very good.

<span style="color:purple">[Q]: did you expect the runtime to be similar? Can you come up with a potential explenation?</span>

[A]:

Let's now find out how many variants were called. If you remember, the vcf file is build up with a header (starting with ## marks), a colum title row starting with # and variant lines. So, we can find out the number of variants by counting the lines NOT starting with #. Linux has a very convenient tool for that called **grep**: it can find text but also count lines containing (or in our case not containing) a string. The parameters are **-c** for counting and **-i** for inverse (so counting all lines that did not contain the string). After the options, provide the string to match (**'#'**) and of course you need to specify the vcf file you want to search through. You can also use **\*.vcf** to look through all vcf files.

<span style="color:green">[DO]: count the variants in both WES and WGS vcf files.</span>

<span style="color:purple">[Q:]: explain the difference in variantnumbers between the WES and WGS datasets.</span>

[A]:

### 4B. Variant Filtering <a name="Variant Filtering"></a>

Eventhough the GATK has a pretty sofisticated Variantcaller, you still need to determine if your variants are of acceptable quality and try to filter out noise from the sequencing. We can do this by applying filtering logic on characteristics of the variants, say coverage or quality for instance. Within the VCF format there is a special colomn reserved that shows the results of filtering rules: either PASS is everything is well, or if a variant fails a specific rule, it displays this filter (or multiple).

Depending on your application, the need and 'weight' of filters can differ: if you want to be supersensitive, don't filter too strictly. If you want to be very precise, filter rigourously. However, we do want to keep all the information in the vcf file, so we annotate the filterstatus in the appropriate way (filtersettings and names go into the header of the VCF, and the filter results go in the filter column). This way, results are very traceable since there is documentation on what analyses are beeing done.(We need to be FAIR remember :)).

You do need to take into account in subsequent analysis that the filter status is correctly used. Many tools allow for using 'PASS' variants only, or 'ALL' variants if you need to ignore the filterstate.

Moreover, SNPs and Indels require different filters, since they have different characteristics. To apply this correctly, we first need to split the vcf file into a vcf containing only SNPs and one containing only INDELs by using **SelectVariants (-T**). We can then define filter rules and apply them on the appropriate vcf file. When this is done, we simpely merge the filtered VCF files together.

```bash
java -jar $GATK \
-T SelectVariants \
-R $the_genome_reference_file \
-selectType [SNP/INDEL] \
-V $input_file.vcf \
-o $outputfile.vcf
```


<span style="color:green">[DO]: generate two seperate files with SNPs and INDELs from the **WES** vcf files using the instructions above. </span>

<span style="color:green">[DO]: generate two seperate files with SNPs and INDELs from the **WGS** vcf files using the instructions above. </span>

Now we have seperated the SNPs and the INDELs we can apply the filterrules. Since there are many possibilities it is not easy to formulate proper filter rules and it can take quite a bit of tuning to define filters that are neither too strict or too loose. We will take the advice from the [GATK's best practise guidelines](https://software.broadinstitute.org/gatk/best-practices/bp_3step.php?case=GermShortWGS) for this excersize.

The GATK filtering tools is called **VariantFiltration (-T )**, and next to the standard genome reference file (**-R**), vcf inputfile (**-V**) and an output filename (**-o**) it need paired **--filterExpression** and **--filterName** parameters. You can specify multiple if needed and each expression is referred to by it's name.

For SNPs the best practises advises: "QD < 2.0 || FS > 60.0 || MQ < 40.0 || MQRankSum < -1.0 || ReadPosRankSum < -8.0"

For INDELs this is: "QD < 2.0 || FS > 200.0 || ReadPosRankSum < -20.0"

<span style="color:green">[DO]: Apply the appropriate filters on the WES and WGS SNP and INDEL vcf files; you can define you're own name for the filters. Make sure the output files clearly mention that variants are "filtered".


In [ ]:
WES:

In [ ]:
WGS:

<span style="color:purple">[Q:]: How many SNPs and INDELs do we have for echt dataset? Remember you can use 'grep' to count variant lines in a vcf file...</span>

[A]:

<span style="color:purple">[Q:]: How many **PASS** SNPs and INDELs do we have for echt dataset? Remember you can use 'grep' to count variant lines in a vcf file...</span>

[A]:

In this stage we have the variants filtered, but to keep working with sepperate SNP and INDEL vcf files is not very convenient. We can simple merge the two vcf files by using GATK's tool **CombineVariants (-T)**. As usual you need to specify the reference (**-R**), and now multiple input vcf files (**-V**). Take care not to mix the WES and WGS files!

<span style="color:green">[DO]: merge the SNP and INDEL vcf files together in a new vcf for both the WES and WGS datasets. Use the grep instruction to count the variants again in each resulting vcf file and check with your earlier count if you still have the same variantcounts.</span>

In [ ]:
WES:

In [ ]:
WGS:

### 4C. Variant annotation <a name="Variant Annotation"></a>

The VCF files we now have contain mostly technical information on the sequencing and variant quality. You could say the only biological information is the genotype. Run the code below, where you change the COMPUTERNAMEHERE in the name of your server (in the adressbar of the browser between https:// and .amazonaws.com).  If you click on the link for a vcf file and open it to view it contents. If you need help 'decoding' the vcf, you can review the [VCF format specifications](https://samtools.github.io/hts-specs/VCFv4.2.pdf)

In [ ]:
for i in *.vcf; 
    do echo https://COMPUTERNAMEHERE.amazonaws.com:8888/files/$i;
done

For most people, the information from the vcf file is not immediately useful and additional biological information needs to be added. We call this annotation: we retrieve information from other sources about a variant and transfer data into the vcf file. This annotation can be diverse: from occurence of a variant in public databases, predicted impact on transcription or translation to very specific details. The format of the vcf file allows storing of annotation in the INFO section and many tools have the capabilities to add this annotation from any other structured file (vcf, text table etc) to a vcf.

In the next excersizes we will annotate our vcf files with:
- The occurence of the variant in the most well known SNP database [dbSNP](https://www.ncbi.nlm.nih.gov/projects/SNP)
- Annotation and prediction of the effects of variants on genes (such as amino acid changes etc)